In [0]:
import os

# Define the base directory
BASE_DIR = "/Workspace/Users/douglas.moore@databricks.com/ols-mcp-server"
os.makedirs(BASE_DIR, exist_ok=True)

# 1. Create app.py
app_py = '''from fastapi import FastAPI, HTTPException, Request
from fastapi.responses import JSONResponse
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel
from typing import Optional, Dict, Any, List
import httpx
import logging
from contextlib import asynccontextmanager

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

OLS_API_BASE_URL = "https://www.ebi.ac.uk/ols4/api"

@asynccontextmanager
async def lifespan(app: FastAPI):
    logger.info("Starting OLS MCP FastAPI Service")
    yield
    logger.info("Shutting down OLS MCP FastAPI Service")

app = FastAPI(
    title="OLS MCP Service",
    description="Databricks Apps-based MCP service for Ontology Lookup Service (OLS) API",
    version="1.0.0",
    lifespan=lifespan
)

app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

class SearchRequest(BaseModel):
    query: str
    ontology: Optional[str] = None
    type: Optional[str] = None
    rows: Optional[int] = 10

class TermRequest(BaseModel):
    ontology: str
    term_id: str

class OntologyRequest(BaseModel):
    ontology_id: str

@app.get("/")
async def root():
    return {
        "service": "OLS MCP Service",
        "status": "healthy",
        "version": "1.0.0",
        "description": "Databricks Apps-based MCP service for Ontology Lookup Service"
    }

@app.get("/health")
async def health_check():
    try:
        async with httpx.AsyncClient() as client:
            response = await client.get(f"{OLS_API_BASE_URL}/ontologies", timeout=5.0)
            ols_status = "healthy" if response.status_code == 200 else "degraded"
    except Exception as e:
        logger.error(f"Health check failed: {e}")
        ols_status = "unhealthy"
    
    return {
        "status": "healthy",
        "ols_api_status": ols_status
    }

@app.post("/mcp/search")
async def search_ontologies(request: SearchRequest):
    try:
        params = {
            "q": request.query,
            "rows": request.rows
        }
        
        if request.ontology:
            params["ontology"] = request.ontology
        if request.type:
            params["type"] = request.type
        
        async with httpx.AsyncClient() as client:
            response = await client.get(
                f"{OLS_API_BASE_URL}/search",
                params=params,
                timeout=30.0
            )
            response.raise_for_status()
            
        return response.json()
    
    except httpx.HTTPError as e:
        logger.error(f"Search failed: {e}")
        raise HTTPException(status_code=500, detail=f"OLS API error: {str(e)}")

@app.get("/mcp/ontologies")
async def list_ontologies():
    try:
        async with httpx.AsyncClient() as client:
            response = await client.get(
                f"{OLS_API_BASE_URL}/ontologies",
                timeout=30.0
            )
            response.raise_for_status()
            
        return response.json()
    
    except httpx.HTTPError as e:
        logger.error(f"List ontologies failed: {e}")
        raise HTTPException(status_code=500, detail=f"OLS API error: {str(e)}")

@app.post("/mcp/ontology")
async def get_ontology(request: OntologyRequest):
    try:
        async with httpx.AsyncClient() as client:
            response = await client.get(
                f"{OLS_API_BASE_URL}/ontologies/{request.ontology_id}",
                timeout=30.0
            )
            response.raise_for_status()
            
        return response.json()
    
    except httpx.HTTPError as e:
        logger.error(f"Get ontology failed: {e}")
        raise HTTPException(status_code=500, detail=f"OLS API error: {str(e)}")

@app.post("/mcp/term")
async def get_term(request: TermRequest):
    try:
        async with httpx.AsyncClient() as client:
            import urllib.parse
            encoded_term = urllib.parse.quote(request.term_id, safe='')
            
            response = await client.get(
                f"{OLS_API_BASE_URL}/ontologies/{request.ontology}/terms/{encoded_term}",
                timeout=30.0
            )
            response.raise_for_status()
            
        return response.json()
    
    except httpx.HTTPError as e:
        logger.error(f"Get term failed: {e}")
        raise HTTPException(status_code=500, detail=f"OLS API error: {str(e)}")

@app.get("/mcp/ontologies/{ontology_id}/terms")
async def get_ontology_terms(
    ontology_id: str,
    page: int = 0,
    size: int = 20
):
    try:
        async with httpx.AsyncClient() as client:
            response = await client.get(
                f"{OLS_API_BASE_URL}/ontologies/{ontology_id}/terms",
                params={"page": page, "size": size},
                timeout=30.0
            )
            response.raise_for_status()
            
        return response.json()
    
    except httpx.HTTPError as e:
        logger.error(f"Get ontology terms failed: {e}")
        raise HTTPException(status_code=500, detail=f"OLS API error: {str(e)}")

@app.post("/mcp/initialize")
async def mcp_initialize(request: Request):
    body = await request.json()
    return {
        "protocolVersion": "2024-11-05",
        "capabilities": {
            "tools": {
                "listChanged": False
            }
        },
        "serverInfo": {
            "name": "ols-mcp-server",
            "version": "1.0.0"
        }
    }

@app.post("/mcp/tools/list")
async def mcp_list_tools():
    return {
        "tools": [
            {
                "name": "search_ontologies",
                "description": "Search across ontologies using OLS API",
                "inputSchema": {
                    "type": "object",
                    "properties": {
                        "query": {"type": "string", "description": "Search term"},
                        "ontology": {"type": "string", "description": "Optional specific ontology"},
                        "type": {"type": "string", "description": "Optional type filter"},
                        "rows": {"type": "integer", "description": "Number of results", "default": 10}
                    },
                    "required": ["query"]
                }
            },
            {
                "name": "list_ontologies",
                "description": "List all available ontologies",
                "inputSchema": {"type": "object", "properties": {}}
            },
            {
                "name": "get_ontology",
                "description": "Get details about a specific ontology",
                "inputSchema": {
                    "type": "object",
                    "properties": {
                        "ontology_id": {"type": "string", "description": "Ontology identifier"}
                    },
                    "required": ["ontology_id"]
                }
            },
            {
                "name": "get_term",
                "description": "Get details about a specific term",
                "inputSchema": {
                    "type": "object",
                    "properties": {
                        "ontology": {"type": "string", "description": "Ontology identifier"},
                        "term_id": {"type": "string", "description": "Term IRI or ID"}
                    },
                    "required": ["ontology", "term_id"]
                }
            }
        ]
    }

@app.post("/mcp/tools/call")
async def mcp_call_tool(request: Request):
    body = await request.json()
    tool_name = body.get("name")
    arguments = body.get("arguments", {})
    
    try:
        if tool_name == "search_ontologies":
            result = await search_ontologies(SearchRequest(**arguments))
        elif tool_name == "list_ontologies":
            result = await list_ontologies()
        elif tool_name == "get_ontology":
            result = await get_ontology(OntologyRequest(**arguments))
        elif tool_name == "get_term":
            result = await get_term(TermRequest(**arguments))
        else:
            raise HTTPException(status_code=404, detail=f"Tool not found: {tool_name}")
        
        return {
            "content": [
                {
                    "type": "text",
                    "text": str(result)
                }
            ]
        }
    except Exception as e:
        logger.error(f"Tool execution failed: {e}")
        return {
            "content": [
                {
                    "type": "text",
                    "text": f"Error: {str(e)}"
                }
            ],
            "isError": True
        }

if __name__ == "__main__":
    import uvicorn
    uvicorn.run(app, host="0.0.0.0", port=8000)
'''

with open(os.path.join(BASE_DIR, "app.py"), "w") as f:
    f.write(app_py)

# 2. Create app.yaml
app_yaml = '''name: ols-mcp-service
description: "MCP service for Ontology Lookup Service (OLS) API"

compute:
  - name: default
    spec:
      instance_profile: "GENERAL_PURPOSE"
      node_type_id: "i3.xlarge"
      num_workers: 0

app:
  command:
    - "uvicorn"
    - "app:app"
    - "--host"
    - "0.0.0.0"
    - "--port"
    - "8000"
  
  env:
    - name: OLS_API_BASE_URL
      value: "https://www.ebi.ac.uk/ols4/api"
    - name: LOG_LEVEL
      value: "INFO"
    - name: PYTHONUNBUFFERED
      value: "1"
  
  port: 8000
  
  health_check:
    path: "/health"
    interval_seconds: 30
    timeout_seconds: 10
    healthy_threshold: 2
    unhealthy_threshold: 3

resources:
  - name: requirements
    source: requirements.txt
  - name: app
    source: app.py

permissions:
  - level: CAN_MANAGE
    group_name: "admins"
  - level: CAN_VIEW
    group_name: "users"
'''

with open(os.path.join(BASE_DIR, "app.yaml"), "w") as f:
    f.write(app_yaml)

# 3. Create requirements.txt
requirements = '''fastapi>=0.115.0
uvicorn[standard]>=0.32.0
httpx>=0.24.0
pydantic>=2.0.0
python-multipart>=0.0.9
python-json-logger>=2.0.7
fastmcp>=2.10.5
'''

with open(os.path.join(BASE_DIR, "requirements.txt"), "w") as f:
    f.write(requirements)

# 4. Create DEPLOYMENT.md
deployment = '''# Deploying OLS MCP Service to Databricks Apps

## Prerequisites
1. Databricks workspace with Apps enabled
2. Databricks CLI installed and configured
3. Appropriate permissions to create and manage apps

## Deployment Steps

### Using Databricks CLI
bash
databricks apps create ols-mcp-service
databricks apps deploy ols-mcp-service /Workspace/Users/douglas.moore@databricks.com/ols-mcp-server
databricks apps start ols-mcp-service
databricks apps get ols-mcp-service


## Testing
bash
curl https://<your-app-url>/health
curl https://<your-app-url>/mcp/ontologies


## API Documentation
- Swagger UI: https://<your-app-url>/docs
- ReDoc: https://<your-app-url>/redoc
'''

with open(os.path.join(BASE_DIR, "DEPLOYMENT.md"), "w") as f:
    f.write(deployment)

# Verify
print("=" * 80)
print("All files created successfully!")
print("=" * 80)
for filename in ["app.py", "app.yaml", "requirements.txt", "DEPLOYMENT.md"]:
    filepath = os.path.join(BASE_DIR, filename)
    size = os.path.getsize(filepath)
    print(f"{filename:20s} - {size:,} bytes")
print("=" * 80)
print(f"\nLocation: {BASE_DIR}")
print("\nNext: Read DEPLOYMENT.md for deployment instructions")

In [0]:
import os
import shutil

# Define paths
BASE_DIR = "/Workspace/Users/douglas.moore@databricks.com/ols-mcp-server"
APP_DIR = os.path.join(BASE_DIR, "app")

# Create app directory structure
os.makedirs(APP_DIR, exist_ok=True)

print(f"Creating DAB structure in {APP_DIR}...\n")

# 1. Create databricks.yml (DAB configuration)
databricks_yml = '''bundle:
  name: ols-mcp-service

include:
  - resources/*.yml

targets:
  dev:
    mode: development
    default: true
    workspace:
      host: {{workspace.host}}
  
  prod:
    mode: production
    workspace:
      host: {{workspace.host}}
      root_path: /Workspace/Users/{{workspace.current_user.userName}}/.bundle/ols-mcp-service/prod
'''

with open(os.path.join(BASE_DIR, "databricks.yml"), "w") as f:
    f.write(databricks_yml)

print("✅ Created databricks.yml")

# 2. Create resources directory and app.yml
resources_dir = os.path.join(BASE_DIR, "resources")
os.makedirs(resources_dir, exist_ok=True)

app_resource_yml = '''resources:
  apps:
    ols_mcp_service:
      name: ols-mcp-service
      description: "MCP service for Ontology Lookup Service (OLS) API"
      
      resources:
        - name: app_code
          source_path: ./app
      
      config:
        command:
          - "uvicorn"
          - "app:app"
          - "--host"
          - "0.0.0.0"
          - "--port"
          - "8000"
        
        env:
          - name: OLS_API_BASE_URL
            value: "https://www.ebi.ac.uk/ols4/api"
          - name: LOG_LEVEL
            value: "INFO"
          - name: PYTHONUNBUFFERED
            value: "1"
'''

with open(os.path.join(resources_dir, "app.yml"), "w") as f:
    f.write(app_resource_yml)

print("✅ Created resources/app.yml")

# 3. Copy app files to ./app directory
app_files = ["app.py", "requirements.txt"]

for filename in app_files:
    src = os.path.join(BASE_DIR, filename)
    dst = os.path.join(APP_DIR, filename)
    if os.path.exists(src):
        shutil.copy2(src, dst)
        print(f"✅ Copied {filename} to app/")
    else:
        print(f"⚠️  Warning: {filename} not found at {src}")

# 4. Create .gitignore
gitignore = '''.databricks/
__pycache__/
*.pyc
*.pyo
*.egg-info/
.DS_Store
.venv/
venv/
'''

with open(os.path.join(BASE_DIR, ".gitignore"), "w") as f:
    f.write(gitignore)

print("✅ Created .gitignore")

# 5. Create README for DAB deployment
readme_dab = '''# OLS MCP Service - Databricks Asset Bundle

This directory contains a Databricks Asset Bundle (DAB) configuration for deploying the OLS MCP service as a Databricks App.

## Directory Structure

```
ols-mcp-server/
├── databricks.yml          # Main DAB configuration
├── resources/
│   └── app.yml            # App resource definition
├── app/
│   ├── app.py             # FastAPI application
│   └── requirements.txt   # Python dependencies
└── README_DAB.md          # This file
```

## Prerequisites

1. Install Databricks CLI:
   ```bash
   pip install databricks-cli
   ```

2. Authenticate with your workspace:
   ```bash
   databricks auth login --host <your-workspace-url>
   ```

## Deployment

### Deploy to Development

```bash
cd /Workspace/Users/douglas.moore@databricks.com/ols-mcp-server
databricks bundle deploy
```

This will:
- Validate the bundle configuration
- Upload app files to the workspace
- Create/update the Databricks App
- Deploy to the `dev` target (default)

### Deploy to Production

```bash
databricks bundle deploy --target prod
```

### View Deployment Status

```bash
databricks bundle run ols_mcp_service
```

### Destroy/Remove the App

```bash
databricks bundle destroy
```

## Validate Configuration

Before deploying, validate your bundle:

```bash
databricks bundle validate
```

## Local Development

To test the FastAPI app locally:

```bash
cd app
pip install -r requirements.txt
uvicorn app:app --reload
```

Access the API at: http://localhost:8000
- Swagger UI: http://localhost:8000/docs
- ReDoc: http://localhost:8000/redoc

## Configuration

### Environment Variables

Edit `resources/app.yml` to modify environment variables:
- `OLS_API_BASE_URL`: Base URL for OLS API
- `LOG_LEVEL`: Logging level (INFO, DEBUG, WARNING, ERROR)

### Targets

The bundle supports two deployment targets:
- `dev` (default): Development environment
- `prod`: Production environment

## Troubleshooting

### View App Logs

```bash
databricks apps logs ols-mcp-service
```

### Check App Status

```bash
databricks apps get ols-mcp-service
```

### Common Issues

1. **Authentication Error**: Run `databricks auth login` again
2. **Bundle Validation Failed**: Check `databricks.yml` syntax
3. **App Won\'t Start**: Check logs for dependency or runtime errors

## API Endpoints

Once deployed, your app will expose:

- `GET /` - Health check
- `GET /health` - Detailed health status
- `POST /mcp/search` - Search ontologies
- `GET /mcp/ontologies` - List all ontologies
- `POST /mcp/ontology` - Get ontology details
- `POST /mcp/term` - Get term details
- `GET /mcp/ontologies/{id}/terms` - Get ontology terms
- `POST /mcp/initialize` - MCP protocol initialization
- `POST /mcp/tools/list` - List available MCP tools
- `POST /mcp/tools/call` - Execute MCP tool

## Next Steps

1. Review and customize `databricks.yml` and `resources/app.yml`
2. Run `databricks bundle validate` to check configuration
3. Deploy with `databricks bundle deploy`
4. Test the deployed app endpoints
5. Monitor logs and performance
'''

with open(os.path.join(BASE_DIR, "README_DAB.md"), "w") as f:
    f.write(readme_dab)

print("✅ Created README_DAB.md")

# Summary
print("\n" + "=" * 80)
print("DAB Structure Created Successfully!")
print("=" * 80)
print(f"\nBase Directory: {BASE_DIR}")
print(f"App Directory: {APP_DIR}")
print("\nFiles created:")
print("  📄 databricks.yml          - Main DAB configuration")
print("  📄 resources/app.yml       - App resource definition")
print("  📁 app/                    - Application code directory")
print("  📄 app/app.py              - FastAPI application")
print("  📄 app/requirements.txt    - Python dependencies")
print("  📄 .gitignore              - Git ignore rules")
print("  📄 README_DAB.md           - DAB deployment guide")
print("\n" + "=" * 80)
print("\n🚀 Next Steps:")
print("  1. Review databricks.yml and resources/app.yml")
print("  2. Run: databricks bundle validate")
print("  3. Deploy: databricks bundle deploy")
print("  4. Check status: databricks apps get ols-mcp-service")
print("=" * 80)

In [0]:
import os
import shutil
import yaml

# Define paths
BASE_DIR = "/Workspace/Users/douglas.moore@databricks.com/ols-mcp-server"
APP_DIR = os.path.join(BASE_DIR, "app")

print("=" * 80)
print("Fixing and Validating DAB Structure")
print("=" * 80)

# 1. Copy missing files to app directory
print("\n1. Copying application files to ./app directory...")
app_files = ["app.py", "requirements.txt"]

for filename in app_files:
    src = os.path.join(BASE_DIR, filename)
    dst = os.path.join(APP_DIR, filename)
    if os.path.exists(src):
        shutil.copy2(src, dst)
        print(f"   ✅ Copied {filename} to app/ ({os.path.getsize(dst):,} bytes)")
    else:
        print(f"   ❌ ERROR: {filename} not found at {src}")

# 2. Validate databricks.yml
print("\n2. Validating databricks.yml...")
databricks_yml_path = os.path.join(BASE_DIR, "databricks.yml")
if os.path.exists(databricks_yml_path):
    with open(databricks_yml_path, 'r') as f:
        content = f.read()
        print(f"   ✅ File exists ({len(content)} bytes)")
        print("   Content preview:")
        print("   " + "\n   ".join(content.split('\n')[:10]))
        
        # Check for required fields
        if 'bundle:' in content and 'name:' in content:
            print("   ✅ Contains required bundle configuration")
        else:
            print("   ⚠️  WARNING: Missing required bundle fields")
        
        if 'targets:' in content:
            print("   ✅ Contains deployment targets")
        else:
            print("   ⚠️  WARNING: Missing deployment targets")
else:
    print(f"   ❌ ERROR: databricks.yml not found")

# 3. Validate resources/app.yml
print("\n3. Validating resources/app.yml...")
app_yml_path = os.path.join(BASE_DIR, "resources", "app.yml")
if os.path.exists(app_yml_path):
    with open(app_yml_path, 'r') as f:
        content = f.read()
        print(f"   ✅ File exists ({len(content)} bytes)")
        print("   Content preview:")
        print("   " + "\n   ".join(content.split('\n')[:15]))
        
        # Check for required fields
        if 'resources:' in content and 'apps:' in content:
            print("   ✅ Contains app resource definition")
        else:
            print("   ⚠️  WARNING: Missing required resource fields")
        
        if 'source_path: ./app' in content:
            print("   ✅ Correctly references ./app directory")
        else:
            print("   ⚠️  WARNING: source_path may be incorrect")
else:
    print(f"   ❌ ERROR: resources/app.yml not found")

# 4. Validate app directory contents
print("\n4. Validating ./app directory contents...")
if os.path.exists(APP_DIR):
    app_contents = os.listdir(APP_DIR)
    print(f"   ✅ Directory exists with {len(app_contents)} files")
    
    required_files = ['app.py', 'requirements.txt']
    for req_file in required_files:
        file_path = os.path.join(APP_DIR, req_file)
        if os.path.exists(file_path):
            size = os.path.getsize(file_path)
            print(f"   ✅ {req_file:20s} - {size:,} bytes")
        else:
            print(f"   ❌ MISSING: {req_file}")
else:
    print(f"   ❌ ERROR: ./app directory not found")

# 5. Check for common issues
print("\n5. Checking for common configuration issues...")

# Check if app.py has the correct structure
app_py_path = os.path.join(APP_DIR, "app.py")
if os.path.exists(app_py_path):
    with open(app_py_path, 'r') as f:
        app_content = f.read()
        
    checks = [
        ('FastAPI import', 'from fastapi import FastAPI'),
        ('App instance', 'app = FastAPI('),
        ('Health endpoint', '@app.get("/health")'),
        ('MCP endpoints', '/mcp/'),
    ]
    
    for check_name, check_str in checks:
        if check_str in app_content:
            print(f"   ✅ {check_name} found")
        else:
            print(f"   ⚠️  WARNING: {check_name} not found")
else:
    print("   ❌ Cannot validate app.py - file not found")

# Check requirements.txt
req_path = os.path.join(APP_DIR, "requirements.txt")
if os.path.exists(req_path):
    with open(req_path, 'r') as f:
        req_content = f.read()
    
    required_deps = ['fastapi', 'uvicorn', 'httpx', 'pydantic']
    for dep in required_deps:
        if dep in req_content.lower():
            print(f"   ✅ Dependency '{dep}' found")
        else:
            print(f"   ⚠️  WARNING: Dependency '{dep}' not found")
else:
    print("   ❌ Cannot validate requirements.txt - file not found")

# 6. Summary
print("\n" + "=" * 80)
print("Validation Summary")
print("=" * 80)

all_files = [
    ("databricks.yml", os.path.join(BASE_DIR, "databricks.yml")),
    ("resources/app.yml", os.path.join(BASE_DIR, "resources", "app.yml")),
    ("app/app.py", os.path.join(APP_DIR, "app.py")),
    ("app/requirements.txt", os.path.join(APP_DIR, "requirements.txt")),
    (".gitignore", os.path.join(BASE_DIR, ".gitignore")),
    ("README_DAB.md", os.path.join(BASE_DIR, "README_DAB.md")),
]

all_valid = True
for name, path in all_files:
    if os.path.exists(path):
        size = os.path.getsize(path)
        print(f"✅ {name:25s} - {size:,} bytes")
    else:
        print(f"❌ {name:25s} - MISSING")
        all_valid = False

print("\n" + "=" * 80)
if all_valid:
    print("✅ All files present and validated!")
    print("\n🚀 Ready to deploy with: databricks bundle deploy")
else:
    print("⚠️  Some files are missing or invalid. Please review the errors above.")
print("=" * 80)

In [0]:
import os
import re

BASE_DIR = "/Workspace/Users/douglas.moore@databricks.com/ols-mcp-server"

print("=" * 80)
print("Final DAB Configuration Review")
print("=" * 80)

# 1. Review databricks.yml for Databricks Apps compatibility
print("\n1. Reviewing databricks.yml for Databricks Apps...")
databricks_yml_path = os.path.join(BASE_DIR, "databricks.yml")
with open(databricks_yml_path, 'r') as f:
    dab_content = f.read()

print("\nCurrent databricks.yml:")
print("-" * 40)
print(dab_content)
print("-" * 40)

# Check for issues
issues = []
recommendations = []

if '{{workspace.host}}' in dab_content:
    print("✅ Uses workspace variables correctly")
else:
    issues.append("Missing workspace.host variable")

if 'include:' in dab_content and 'resources/*.yml' in dab_content:
    print("✅ Includes resources directory")
else:
    issues.append("Missing resources include")

# 2. Review resources/app.yml
print("\n2. Reviewing resources/app.yml...")
app_yml_path = os.path.join(BASE_DIR, "resources", "app.yml")
with open(app_yml_path, 'r') as f:
    app_yml_content = f.read()

print("\nCurrent resources/app.yml:")
print("-" * 40)
print(app_yml_content)
print("-" * 40)

# Check app.yml structure
if 'resources:' in app_yml_content and 'apps:' in app_yml_content:
    print("✅ Correct resource type (apps)")
else:
    issues.append("Incorrect resource structure")

if 'source_path: ./app' in app_yml_content:
    print("✅ Source path points to ./app")
else:
    issues.append("Source path not configured correctly")

if 'uvicorn' in app_yml_content:
    print("✅ Uses uvicorn as ASGI server")
else:
    issues.append("Missing uvicorn command")

# 3. Check for potential issues
print("\n3. Checking for potential deployment issues...")

# Check if port is specified
if 'port:' in app_yml_content or '8000' in app_yml_content:
    print("✅ Port configuration found")
    recommendations.append("Verify that port 8000 is not blocked in your workspace")
else:
    issues.append("Port not specified in configuration")

# Check environment variables
if 'OLS_API_BASE_URL' in app_yml_content:
    print("✅ OLS API URL configured")
else:
    issues.append("Missing OLS_API_BASE_URL environment variable")

# 4. Verify app directory structure
print("\n4. Verifying app directory structure...")
app_dir = os.path.join(BASE_DIR, "app")
if os.path.exists(app_dir):
    files = os.listdir(app_dir)
    print(f"   Files in ./app: {', '.join(files)}")
    
    if 'app.py' in files:
        app_py_path = os.path.join(app_dir, 'app.py')
        with open(app_py_path, 'r') as f:
            app_py_content = f.read()
        
        # Check if app variable is defined
        if re.search(r'app\s*=\s*FastAPI\(', app_py_content):
            print("✅ FastAPI app instance named 'app' (matches uvicorn app:app)")
        else:
            issues.append("FastAPI app instance may not be named 'app'")
        
        # Check if it's async
        if 'async def' in app_py_content:
            print("✅ Uses async endpoints (good for I/O operations)")
        
        # Check for __main__ block
        if 'if __name__ == "__main__"' in app_py_content:
            print("✅ Has __main__ block for local testing")
    else:
        issues.append("app.py not found in ./app directory")
    
    if 'requirements.txt' in files:
        print("✅ requirements.txt present")
    else:
        issues.append("requirements.txt not found in ./app directory")
else:
    issues.append("./app directory not found")

# 5. Recommendations
print("\n" + "=" * 80)
print("Configuration Analysis")
print("=" * 80)

if issues:
    print("\n⚠️  Issues Found:")
    for i, issue in enumerate(issues, 1):
        print(f"   {i}. {issue}")
else:
    print("\n✅ No critical issues found!")

print("\n💡 Recommendations:")
default_recommendations = [
    "Test the app locally first: cd app && uvicorn app:app --reload",
    "Validate bundle before deploying: databricks bundle validate",
    "Start with dev target: databricks bundle deploy (uses dev by default)",
    "Monitor logs after deployment: databricks apps logs ols-mcp-service",
    "Check app status: databricks apps get ols-mcp-service",
    "Access Swagger UI at: https://<app-url>/docs",
    "Ensure your workspace has internet access to reach https://www.ebi.ac.uk/ols4/api",
]

for i, rec in enumerate(default_recommendations + recommendations, 1):
    print(f"   {i}. {rec}")

print("\n" + "=" * 80)
print("🎯 Final Status")
print("=" * 80)

if not issues:
    print("✅ Configuration is valid and ready for deployment!")
    print("\n🚀 Next command: databricks bundle deploy")
else:
    print("⚠️  Please fix the issues above before deploying.")

print("=" * 80)